In [ ]:
from scipy.spatial import distance as dist
from playsound import playsound
from imutils import face_utils
import mediapipe as mp
from pygame import *
import numpy as np
import imutils
import dlib
import time
import cv2
import os

In [ ]:
# EYE
EYE_AR_CONSEC_FRAMES = 30
alarm_drowinsess=False
EYE_AR_THRESH = 0.3
COUNTER = 0
# YAWN
YAWN_THRESH_farme = 10
alarm_yawn=False
YAWN_THRESH = 20
count_YAWN=0
# HEADPOSE
alarm_head_pose=False
Head_Pose_Frame = 25
count_headpose = 0

In [ ]:
mixer.init()
def playsound():
    
    mixer.music.load('wakeup.mp3')
    mixer.music.play()


def stopsound():
    mixer.music.stop()
    
    
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])    
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear


def final_ear(shape):
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
    leftEye = shape[lStart:lEnd]
    rightEye = shape[rStart:rEnd]
    leftEAR = eye_aspect_ratio(leftEye)
    rightEAR = eye_aspect_ratio(rightEye)
    ear = (leftEAR + rightEAR) / 2.0
    return (ear, leftEye, rightEye)


def lip_distance(shape):
    top_lip = shape[50:53]
    top_lip = np.concatenate((top_lip, shape[61:64]))
    low_lip = shape[56:59]
    low_lip = np.concatenate((low_lip, shape[65:68]))   
    top_mean = np.mean(top_lip, axis=0)
    low_mean = np.mean(low_lip, axis=0)
    distance = abs(top_mean[1] - low_mean[1])
    return distance



In [4]:
#Head Pose
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence = 0.5 , min_tracking_confidence = 0.5)
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1,circle_radius=1)

In [5]:
print("-> Loading the predictor and detector...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
print("-> Starting Video Stream")

-> Loading the predictor and detector...
-> Starting Video Stream


In [ ]:
vs = cv2.VideoCapture(0)
while vs.isOpened():
    r, frame = vs.read()
    frame = imutils.resize(frame, width=700)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    frame = cv2.cvtColor(cv2.flip(frame,1),cv2.COLOR_BGR2RGB)
    
    frame.flags.writeable = False
    results = face_mesh.process(frame)
    img_h,img_w,Img_c = frame.shape
    face_3d = []
    face_2d = []
    
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    rects = detector(gray, 0)
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            for idx, lm in enumerate(face_landmarks.landmark):
                if idx == 33 or idx == 263 or idx == 1 or idx == 61 or idx == 291 or idx == 199:
                    if idx == 1:
                        nose_2d = (lm.x * img_w , lm.y * img_h)
                        nose_3d = (lm.x * img_w , lm.y * img_h , lm.z * 3000)
                    
                    x , y = int(lm.x * img_w) , int(lm.y * img_h)
                    face_2d.append([x, y])
                    face_3d.append([x, y, lm.z])
                    
            face_2d = np.array(face_2d, dtype = np.float64)
            face_3d = np.array(face_3d, dtype = np.float64)
            focal_length = 1 * img_w
            cam_matrix = np.array([[focal_length, 0, img_h / 2],
                                  [0 ,focal_length, img_w / 2],
                                  [0 ,0 ,1]])
            
            dist_matrix = np.zeros((4,1),dtype= np.float64) 
            r, rot_vec, trans_vec = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)
            rmat, jac = cv2.Rodrigues(rot_vec)
            angles , mtxR , mtxQ , Qx , Qy , Qz = cv2.RQDecomp3x3(rmat)
            x = angles[0] * 360
            y = angles[1] * 360
            z = angles[2] * 360
            
            if y < -10:
                text = "Looking Left"
                count_headpose = count_headpose + 1
                if count_headpose >= Head_Pose_Frame:
                    if not alarm_head_pose and not alarm_drowinsess and not alarm_yawn:
                        alarm_head_pose=True
                        playsound()
                        print("open alarm of looking left")

                    
            elif y > 10:
                text = "Looking Right"
                count_headpose = count_headpose + 1
                if (count_headpose >= Head_Pose_Frame):
                     if not alarm_head_pose and not alarm_drowinsess and not alarm_yawn:
                        alarm_head_pose=True
                        playsound()
                        print("open alarm of looking Right")
                    
            elif x < -10:
                text = "Looking Down"
                count_headpose = count_headpose + 1
                if (count_headpose >= Head_Pose_Frame):
                     if not alarm_head_pose and not alarm_drowinsess and not alarm_yawn:
                        alarm_head_pose=True
                        playsound()
                        print("open alarm of looking Down")
 
            elif x > 10:
                text = "Looking Up"
                count_headpose = count_headpose + 1
                if (count_headpose >= Head_Pose_Frame):
                     if not alarm_head_pose and not alarm_drowinsess and not alarm_yawn:
                        alarm_head_pose=True
                        playsound()
                        print("open alarm of looking Up")

            else:
                text = "Forward"
                count_headpose=0
                if alarm_head_pose==True:
                    alarm_head_pose=False
                    stopsound()
                    print("stop alarm of head pose")
                
            nose_3d_projection , jacobian = cv2.projectPoints(nose_3d, rot_vec, trans_vec, cam_matrix, dist_matrix)
            p1 = (int(nose_2d[0]), int(nose_2d[1]))
            p2 = (int(nose_2d[0] + y * 10), int(nose_2d[1] - x * 10))
           #cv2.line(frame, p1, p2, (255, 0, 0), 3)
            cv2.putText(frame, text, (10 , 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7 ,(0, 0, 255), 2)
            cv2.putText(frame, "x: " + str(np.round(x,2)), (560, 95), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, "y: " + str(np.round(y,2)), (560, 125), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, "z: " + str(np.round(z,2)), (560, 155), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            mp_drawing.draw_landmarks(
                        image=frame,
                        landmark_list = face_landmarks,
                        connections = mp_face_mesh.FACEMESH_TESSELATION,
                        landmark_drawing_spec = drawing_spec,
                        connection_drawing_spec = drawing_spec)
        #######################################################################################
    
    for rect in rects: 
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        eye = final_ear(shape)
        ear = eye[0]
        leftEye = eye [1]
        rightEye = eye[2]
        distance = lip_distance(shape)
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        #cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        #cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        lip = shape[48:60]
        #cv2.drawContours(frame, [lip], -1, (0, 255, 0), 1)

        if ear < EYE_AR_THRESH :
            COUNTER += 1
            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                if not alarm_drowinsess and not alarm_head_pose and not alarm_yawn:
                    alarm_drowinsess=True
                    playsound()
                    print('open alarm of drwon')

                cv2.putText(frame, "DROWSINESS ALERT!", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        else:
            COUNTER = 0
            if alarm_drowinsess == True:
                alarm_drowinsess=False
                stopsound()
                print('stop alarm of drwon')


        if (distance > YAWN_THRESH):
            count_YAWN+=1
            if (count_YAWN >= YAWN_THRESH_farme):
                if not alarm_yawn and not alarm_head_pose and not alarm_drowinsess:
                    alarm_yawn=True
                    playsound()
                    print('play alarm of yawn')
    
                cv2.putText(frame, "Yawn Alert", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
     
        else:
            count_YAWN=0
            if alarm_yawn== True:
                alarm_yawn=False
                stopsound()
                print('stop alarm of yawn')

        cv2.putText(frame, "EAR: {:.2f}".format(ear), (560, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "YAWN: {:.2f}".format(distance), (560, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
 
        

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("q"):
        break

vs.release()
cv2.destroyAllWindows()
stopsound()